# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 13 2023 10:51AM,264648,16,9568574,Sartorius Lab Products and Service,Released
1,Jun 13 2023 10:51AM,264648,16,9568578,Sartorius Lab Products and Service,Released
2,Jun 13 2023 10:51AM,264648,16,9568611,Sartorius Lab Products and Service,Released
3,Jun 13 2023 10:43AM,264646,10,9566396,Eywa Pharma Inc.,Released
4,Jun 13 2023 10:43AM,264646,10,9566983,Eywa Pharma Inc.,Released
5,Jun 13 2023 10:43AM,264646,10,9567492,Eywa Pharma Inc.,Released
6,Jun 13 2023 10:43AM,264646,10,9567493,Eywa Pharma Inc.,Released
7,Jun 13 2023 10:43AM,264646,10,9568561,Eywa Pharma Inc.,Released
8,Jun 13 2023 10:43AM,264646,10,9568562,Eywa Pharma Inc.,Released
9,Jun 13 2023 10:33AM,264644,21,9568612,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,264641,Released,1
13,264642,Executing,1
14,264644,Released,1
15,264646,Released,6
16,264648,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264641,NaN,1.0
264642,1.0,NaN
264644,NaN,1.0
264646,NaN,6.0
264648,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264559,22.0,6.0
264584,2.0,1.0
264604,1.0,0.0
264620,15.0,3.0
264622,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264559,22,6
264584,2,1
264604,1,0
264620,15,3
264622,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264559,22,6
1,264584,2,1
2,264604,1,0
3,264620,15,3
4,264622,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264559,22,6
1,264584,2,1
2,264604,1,
3,264620,15,3
4,264622,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service
3,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.
9,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc."
10,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC"
11,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc."
12,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc."
15,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service
16,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc."
44,Jun 13 2023 8:56AM,264628,10,Hush Hush
55,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service,,3
1,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,,6
2,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc.",,1
3,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC",,1
4,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc.",1,
5,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",2,1
6,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,,1
7,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",22,6
8,Jun 13 2023 8:56AM,264628,10,Hush Hush,,11
9,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service,3,
1,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,6,
2,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc.",1,
3,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC",1,
4,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc.",,1
5,Jun 13 2023 9:40AM,264584,15,"Alliance Pharma, Inc.",1,2
6,Jun 13 2023 9:40AM,264637,16,Sartorius Lab Products and Service,1,
7,Jun 13 2023 9:34AM,264559,15,"Alliance Pharma, Inc.",6,22
8,Jun 13 2023 8:56AM,264628,10,Hush Hush,11,
9,Jun 13 2023 8:55AM,264627,16,Sartorius Lab Products and Service,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service,3,
1,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,6,
2,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc.",1,
3,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC",1,
4,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service,3.0,NaN
1,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,6.0,NaN
2,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc.",1.0,NaN
3,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC",1.0,NaN
4,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 13 2023 10:51AM,264648,16,Sartorius Lab Products and Service,3.0,0.0
1,Jun 13 2023 10:43AM,264646,10,Eywa Pharma Inc.,6.0,0.0
2,Jun 13 2023 10:33AM,264644,21,"NBTY Global, Inc.",1.0,0.0
3,Jun 13 2023 10:25AM,264641,16,"SHL Pharma, LLC",1.0,0.0
4,Jun 13 2023 10:25AM,264642,21,"NBTY Global, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,793899,17.0,5.0
15,529143,7.0,24.0
16,1323157,6.0,1.0
19,264620,3.0,15.0
21,529286,1.0,1.0
22,264622,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,793899,17.0,5.0
1,15,529143,7.0,24.0
2,16,1323157,6.0,1.0
3,19,264620,3.0,15.0
4,21,529286,1.0,1.0
5,22,264622,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,17.0,5.0
1,15,7.0,24.0
2,16,6.0,1.0
3,19,3.0,15.0
4,21,1.0,1.0
5,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,17.0
1,15,Released,7.0
2,16,Released,6.0
3,19,Released,3.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21,22
Status,,,,,,
Executing,5.0,24.0,1.0,15.0,1.0,1.0
Released,17.0,7.0,6.0,3.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21,22
0,Executing,5.0,24.0,1.0,15.0,1.0,1.0
1,Released,17.0,7.0,6.0,3.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21,22
0,Executing,5.0,24.0,1.0,15.0,1.0,1.0
1,Released,17.0,7.0,6.0,3.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()